In [6]:
path = !(echo $(pwd))

In [8]:
import os
import sys
sys.path.insert(0, os.path.dirname(os.path.dirname(path[0])))

import numpy as np
import chess
import tensorflow
from tensorflow import keras

from src.preprocessing.board_representation import tensor_to_board
from src.evaluation.old_visualisation import *
from src.utils.data_loader import load_train_test
from src.modeling.custom_losses import sum_squared_loss, pc_column_reg, pc_plane_reg, num_pc_reg, custom_regularized_loss

In [9]:
print(tensorflow.__version__)

2.13.0


In [10]:
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

In [11]:
def visualize_tensor(tensor):

    pieces = ["pawn", "knight", "bishop", "rook", "queen", "king"]
    piece_map = ["white "+piece for piece in pieces] + \
        ["black "+piece for piece in pieces] + \
        ["castling rights", "en passant", "turn"]

    def plot_slice(slice_index):
        plt.figure(figsize=(8, 8))
        plt.imshow(tensor[0][:, :, slice_index], cmap='viridis', vmin=0, vmax=1)
        plt.colorbar()
        plt.gca().invert_yaxis()
        plt.title(f"Slice {slice_index} - {piece_map[slice_index]}")
        plt.show()

    return interact(plot_slice, slice_index=IntSlider(min=0, max=tensor[0].shape[2] - 1, step=1, value=0))

In [13]:
train_data, test_data = load_train_test("../../data", "position_tensor_000")
print(len(train_data), len(test_data))

900000 100000


In [14]:
model = keras.models.load_model(
    "../../model/20230918231931_autoencoder.keras",
    custom_objects={
        "sum_squared_loss": sum_squared_loss,
        "num_pc_reg": num_pc_reg,
        "pc_column_reg": pc_column_reg,
        "pc_plane_reg": pc_plane_reg,
        "custom_loss": custom_regularized_loss
    }
)

In [15]:
index = 2

In [16]:
sample = test_data[index].reshape((1,8,8,15))

In [17]:
reconstruction = model.predict_on_batch(sample)

2023-09-24 19:50:42.858034: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fcea0009430 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-09-24 19:50:42.858103: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-09-24 19:50:42.865158: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-24 19:50:42.910544: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [18]:
model.evaluate(x=[reconstruction], y=[sample], return_dict=True)

1/1 [==============================] - 1s 846ms/step - loss: 1.7422 - sum_squared_loss: 1.5000 - num_pc_reg: 0.0000e+00 - pc_column_reg: 1.4297 - pc_plane_reg: 1.0312


{'loss': 1.7421875,
 'sum_squared_loss': 1.5,
 'num_pc_reg': 0.0,
 'pc_column_reg': 1.4296875,
 'pc_plane_reg': 1.03125}

In [19]:
custom_regularized_loss(sample, reconstruction), sum_squared_loss(sample, reconstruction), num_pc_reg(sample, reconstruction), pc_column_reg(sample, reconstruction), pc_plane_reg(sample, reconstruction)

(<tf.Tensor: shape=(), dtype=bfloat16, numpy=1.25>,
 <tf.Tensor: shape=(), dtype=bfloat16, numpy=1.125>,
 <tf.Tensor: shape=(), dtype=bfloat16, numpy=0.00265503>,
 <tf.Tensor: shape=(), dtype=bfloat16, numpy=0.792969>,
 <tf.Tensor: shape=(), dtype=bfloat16, numpy=0.433594>)

In [20]:
binary_reconstruction = binarize_array(reconstruction)

In [21]:
sample_board = tensor_to_board(sample[0])
reconstructed_board = tensor_to_board(binary_reconstruction[0])

In [22]:
print_board(sample_board)

. . . . . . . .
. . . . . . . K
. . . . . . . .
. . . . . P . .
. . . . . . . k
. . . . . . . .
. . q . . . . .
. . . . . . Q .




In [23]:
print_board(reconstructed_board)

. . . . . . . .
. . . . . . . K
. . . . . . . .
. . . . . P . .
. . . . . . . k
. . . . . . . .
. . q . . . . .
. . . . . . . .




In [24]:
visualize_tensor(abs(reconstruction-sample))

interactive(children=(IntSlider(value=0, description='slice_index', max=14), Output()), _dom_classes=('widget-…

<function __main__.visualize_tensor.<locals>.plot_slice(slice_index)>